In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import elasticite as el
import numpy as np
duration = el.get_default_args(el.EdgeGrid.render)['duration']

In [ ]:
import sys
sys.path.append('..')
from scenario_line_fresnel import EdgeGrid
e = EdgeGrid(N_lame=25, grid_type='line')

In [ ]:
e.lames.shape

## Point-Line Distance--2-Dimensional

See http://mathworld.wolfram.com/Point-LineDistance2-Dimensional.html


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
#%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [ ]:
particles = np.array([[.2, .3], [-.5, 1.1], [.3, .9]]).T
fig = plt.figure(figsize=(5, 5))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, 0], particles[1, 0], 'b+')
ax.plot(particles[0, 1:], particles[1, 1:], 'r')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(particles)

projection du point sur la droite

In [ ]:
particles = np.array([[.2, 1.4], [-.5, 1.], [.4, 1.]]).T
particles = np.array([[.2, 1.5], [-.5, .8], [.3, .9]]).T
print(particles)
P = particles[:, 0].copy()
print(P)
print(np.dot(particles[:, 1:].T, np.array([[0, 1], [-1, 0]])))
perp = np.zeros_like(P)
perp[0] = particles[1, 2] - particles[1, 1]
perp[1] = -(particles[0, 2] - particles[0, 1])
print(perp)
# distance to the line
d = perp[0]*(particles[0, 1] - particles[0, 0]) + perp[1]*(particles[1, 1] - particles[1, 0])
print(d, np.sqrt(perp[0]**2 + perp[1]**2), d/np.sqrt(perp[0]**2 + perp[1]**2))
# normalizing
#perp /= np.sqrt((perp**2).sum())
#perp /= np.sqrt(perp[0]**2 + perp[1]**2)
P += d * perp / (perp[0]**2 + perp[1]**2)
fig = plt.figure(figsize=(5, 5))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, 0], particles[1, 0], 'b+')
ax.plot(particles[0, 1:], particles[1, 1:], 'r')
ax.plot(P[0], P[1], 'gs')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(P)

shorter form

In [ ]:
particles = np.array([[.2, 1.4], [-.5, 1.], [.4, 1.]]).T
particles = np.array([[.2, 1.5], [-.5, .8], [.3, .9]]).T
perp = np.array([particles[1, 2] - particles[1, 1], -(particles[0, 2] - particles[0, 1])])
# distance to the line
d = perp[0]*(particles[0, 1] - particles[0, 0]) + perp[1]*(particles[1, 1] - particles[1, 0])
P =  particles[:, 0] + d * perp / (perp**2).sum()

fig = plt.figure(figsize=(5, 5))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, 0], particles[1, 0], 'b+')
ax.plot(particles[0, 1:], particles[1, 1:], 'r')
ax.plot(P[0], P[1], 'gs')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(P)

deducing the mirror image as twice this projection

In [ ]:
particles = np.array([[.2, 1.4], [-.5, 1.], [.4, 1.]]).T
particles = np.array([[.2, 1.5], [-.5, .8], [.3, .9]]).T
perp = np.array([particles[1, 2] - particles[1, 1], -(particles[0, 2] - particles[0, 1])])
# distance to the line
d = perp[0]*(particles[0, 1] - particles[0, 0]) + perp[1]*(particles[1, 1] - particles[1, 0])
P =  particles[:, 0] + 2 *d * perp / (perp**2).sum()

fig = plt.figure(figsize=(5, 5))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, 0], particles[1, 0], 'b+')
ax.plot(particles[0, 1:], particles[1, 1:], 'r')
ax.plot(P[0], P[1], 'gs')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(P)

a function

In [ ]:
particles.T.tolist()

In [ ]:
particles = np.array([[-.5, .5],[.2, 1.5], [-.5, .8], [.3, .9]]).T
particles = np.random.rand(2, 10)
print(particles)
segment = np.array([[-.5, .5], [.5, 1.5]]).T
print(segment)
def mirror(particles, segment, alpha=1.):
    mirror = particles.copy()
    perp = np.array([segment[1][1] - segment[1][0], -(segment[0][1] - segment[0][0])])
    # distance to the line
    d = perp[0]*(segment[0][1] - particles[0, :]) + perp[1]*(segment[1, 1] - particles[1, :])
    mirror[:2, :] =  particles[:2, :] + 2. * d[np.newaxis, :] * perp[:, np.newaxis] / (perp**2).sum()
    if mirror.shape[0]>2: mirror[2, :] =  alpha * particles[2, :]
    #ind = 1-(d == np.zeros_like(d))
    return mirror#[:, ind]

particles_mirror = mirror(particles, segment)

fig = plt.figure(figsize=(5, 5))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, :], particles[1, :], 'b+')
ax.plot(segment[0, :], segment[1, :], 'r')
ax.plot(particles_mirror[0, :], particles_mirror[1, :], 'g+')
ax.plot(np.vstack((particles[0, :], particles_mirror[0, :])), np.vstack((particles[1, :], particles_mirror[1, :])), 'k--')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(particles_mirror)

## miroir d'une image et image d'un miroir

peut on simuler de façon équivalente plusieurs reflections en utilisant le miroir d'un segment

In [ ]:
particles = np.array([[-.5, .5],[.2, 1.5], [-.5, .8], [.3, .9]]).T
particles = np.random.rand(2, 10)
print(particles)
segmentA = np.array([[-.5, .5], [.5, 1.5]]).T
segmentB = np.array([[-0., 0.], [.0, 2.]]).T
print(segmentA, segmentB)

particles_mirrorA = mirror(particles, segmentA)
particles_mirrorAB = mirror(particles_mirrorA, segmentB)
segmentB_mirrorA = mirror(segmentA, segmentB)
particles_mirrorAB_ = mirror(particles, segmentB_mirrorA)

fig = plt.figure(figsize=(15, 15))
border = 0.0
ax = fig.add_axes((border, border, 1.-2*border, 1.-2*border), axisbg='w')
ax.plot(particles[0, :], particles[1, :], 'bo')
ax.plot(segmentA[0, :], segmentA[1, :], 'r')
ax.plot(segmentB[0, :], segmentB[1, :], 'b')
ax.plot(segmentB_mirrorA[0, :], segmentB_mirrorA[1, :], 'b--')
ax.plot(particles_mirrorA[0, :], particles_mirrorA[1, :], 'go')
ax.plot(particles_mirrorAB[0, :], particles_mirrorAB[1, :], 'g+')
ax.plot(particles_mirrorAB_[0, :], particles_mirrorAB_[1, :], 'gx')
ax.plot(np.vstack((particles_mirrorA[0, :], particles_mirrorAB[0, :])), np.vstack((particles_mirrorA[1, :], particles_mirrorAB[1, :])), 'k--')
ax.plot(np.vstack((particles[0, :], particles_mirrorAB_[0, :])), np.vstack((particles[1, :], particles_mirrorAB_[1, :])), 'k-.')
ax.set_xlim([-1, 1])
ax.set_ylim([0, 2])
print(particles_mirror)

# on applique maintenant à la structure

la structure est représentée par une liste de segments sur lesquels on a tiré des points 

In [ ]:
import elasticite as el
import numpy as np

e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure()
fig, ax = e.plot_structure()
print(e.do_structure())

In [ ]:
segments = e.structure_as_segments()
print(segments)

In [ ]:
e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure()
alpha = .8
particles = e.particles.copy()
particles_mirror = particles.copy()
for segment in segments:
    particles_mirror = np.hstack((particles_mirror, mirror(particles, segment, alpha)))
    #print(particles_mirror.shape)#, mirror(e.particles, np.array(segment))).shape)

e.particles = particles_mirror
fig, ax = e.plot_structure()

In [ ]:
alpha *= .8
particles = e.particles.copy()
particles_mirror = particles.copy()
for segment in segments:
    particles_mirror = np.hstack((particles_mirror, mirror(particles, np.array(segment), alpha)))
    #print(particles_mirror.shape)#, mirror(e.particles, np.array(segment))).shape)

e.particles = particles_mirror
fig, ax = e.plot_structure()

In [ ]:
alpha *= .8
particles = e.particles.copy()
particles_mirror = particles.copy()
for segment in segments:
    particles_mirror = np.hstack((particles_mirror, mirror(particles, np.array(segment), alpha)))
    #print(particles_mirror.shape)#, mirror(e.particles, np.array(segment))).shape)

    
e.total_width *= 1.6
e.particles = particles_mirror
fig, ax = e.plot_structure()

Summming up everything in a few functions:


In [ ]:
e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure()
fig, ax = e.plot_structure()

In [ ]:
e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure(N_mirror=1, alpha = .8)
e.total_width *= 1.6
fig, ax = e.plot_structure()

In [ ]:
e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure(N_mirror=2, alpha = .8)
fig, ax = e.plot_structure()

In [ ]:
e = el.EdgeGrid(N_lame=25, grid_type='line')
e.sample_structure(N_mirror=3, alpha = .5)
fig, ax = e.plot_structure()

## git


In [ ]:
!git s
#!git add 2015-11-02\ élasticité\ expansion\ en\ m*

In [ ]:
!git commit -am' expansion - miroir de la structure - principes'

In [ ]:
! git push
